In [ ]:
#crtl f9 inicializar tudo

In [ ]:
pip install pyspark


In [ ]:
pip install fsspec

In [ ]:
pip install gcsfs

Importando funcções padrão. Função Window também importada porque será usado mais na frente

In [ ]:
import os
from google.cloud import storage

from pyspark.sql import SparkSession, Window
spark = SparkSession.builder.appName('GCSFilesRead').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.functions import to_timestamp, to_date

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType,BooleanType,DateType,TimestampType
import pandas as pd


# Crie um esquema para o dataframe

In [ ]:
customSchema = StructType([
   StructField("id_video", StringType(),True),
   StructField("titulo_video", StringType(),True),
   StructField("publicado_em", StringType(),True),
   StructField("id_canal", StringType(),True),
   StructField("nome_canal", StringType(),True),
   StructField("data_destaque", StringType(),True),
   StructField("cont_visualizacao", IntegerType(),True),
   StructField("curtidas", IntegerType(),True),
   StructField("nao_curtidas", IntegerType(),True),
   StructField("cont_comentarios", IntegerType(),True),
   StructField("comentarios_desabilitados", StringType(),True),
   StructField("curtidas_desabilitadas", StringType(),True),
   StructField("pais", StringType(),True),
   StructField("categoria", StringType(),True)

])

schema = customSchema

In [ ]:
import os
serviceaccount = 'gs://projetofinalgrupo8/chave/projetofinalgrupo8-2dcd866c3f46.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

dfspark = pd.read_csv('gs://projetofinalgrupo8/saida/data_tratado_pandas.csv', sep=',',encoding='UTF-8', header=0)
df= spark.createDataFrame(dfspark,schema=schema)

In [ ]:
df.show(5)

+-----------+--------------------+-------------------+--------------------+--------------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+
|   id_video|        titulo_video|       publicado_em|            id_canal|          nome_canal|      data_destaque|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|
+-----------+--------------------+-------------------+--------------------+--------------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|2020-08-11 22:21:49|UCGfBwrCoi9ZJjKiU...|       Pietro Guedes|2020-08-12 00:00:00|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|
|jbGRowa5tIk|ITZY “Not S

In [ ]:
df.printSchema()

root
 |-- id_video: string (nullable = true)
 |-- titulo_video: string (nullable = true)
 |-- publicado_em: string (nullable = true)
 |-- id_canal: string (nullable = true)
 |-- nome_canal: string (nullable = true)
 |-- data_destaque: string (nullable = true)
 |-- cont_visualizacao: integer (nullable = true)
 |-- curtidas: integer (nullable = true)
 |-- nao_curtidas: integer (nullable = true)
 |-- cont_comentarios: integer (nullable = true)
 |-- comentarios_desabilitados: string (nullable = true)
 |-- curtidas_desabilitadas: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- categoria: string (nullable = true)



#CRIAR COLUNAS DE PUBLICAÇÃO DE DATA E DESTAQUE

In [ ]:
#CRIAR COLUNAS DE PUBLICAÇÃO DE DATA
df = df.withColumn("publicado_em_data", F.lit(F.substring('publicado_em', 1, 10)))
df.show(1)

+-----------+--------------------+-------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+
|   id_video|        titulo_video|       publicado_em|            id_canal|   nome_canal|      data_destaque|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|
+-----------+--------------------+-------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|2020-08-11 22:21:49|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|2020-08-12 00:00:00|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs

In [ ]:
#CRIAR COLUNA DE DATA DE DESTAQUE
df = df.withColumn("data_destaque1", F.lit(F.substring('data_destaque', 1, 10)))
df.show(1)

+-----------+--------------------+-------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|   id_video|        titulo_video|       publicado_em|            id_canal|   nome_canal|      data_destaque|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque1|
+-----------+--------------------+-------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|2020-08-11 22:21:49|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|2020-08-12 00:00:00|           263835|   85095|         487|            4500|                    fals

#DROPAR COLUNA

In [ ]:
#DROPAR COLUNA PUBLICADO_EM
df = df.drop('publicado_em')
df.show(1)

+-----------+--------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|      data_destaque|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque1|
+-----------+--------------------+--------------------+-------------+-------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|2020-08-12 00:00:00|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|       2020-08-11|    2020-08-12|


In [ ]:
#DROPAR COLUNA DE DESTAQUE
df = df.drop('data_destaque')
df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque1|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+--------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|       2020-08-11|    2020-08-12|
+-----------+--------------------+--------------------+-------------+-----------

#RENOMEANDO COLUNA DATA DESTAQUE

In [ ]:
df = df.withColumnRenamed("data_destaque1", "data_destaque")
df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|       2020-08-11|   2020-08-12|
+-----------+--------------------+--------------------+-------------+---------------

#RELAÇÃO DE CURTIDAS POR VISUALIZAÇÃO


In [ ]:
df = (df.withColumn("like_por_visualizacao_100", F.round(F.col("curtidas")*100/ F.col("cont_visualizacao"),2))) 

df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|           263835|   85095|         487|            4500|                    false|                 false|  BR|pessoas_e_blogs|       2020-08-11|   2020-08-12|       

#RELAÇÃO DE DESCURTIDAS POR VISUALIZAÇÃO


In [ ]:
df = (df.withColumn("deslike_por_visualizacao_100", F.round(F.col("nao_curtidas")*100/ F.col("cont_visualizacao"),2))) 

df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Guedes|           263835|   85095|         487|            4500|                    fa

#RELAÇÃO DE COMENTARIOS POR VISUALIZAÇÃO + SCHEMA PARA ANALISE DE TYPES


In [ ]:
df = (df.withColumn("coment_por_visualizacao_100", F.round(F.col("cont_comentarios")*100/ F.col("cont_visualizacao"),2))) 

df.show(1)
df.printSchema()

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|coment_por_visualizacao_100|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Gu

#RELAÇÃO DE COMENTARIOS POR VISUALIZAÇÃO


In [ ]:
df = (df.withColumn("coment_por_visualizacao_100", F.round(F.col("cont_comentarios")*100/ F.col("cont_visualizacao"),2))) 

df.show(1)

+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|   id_video|        titulo_video|            id_canal|   nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|coment_por_visualizacao_100|
+-----------+--------------------+--------------------+-------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCoi9ZJjKiU...|Pietro Gu

#TRANSFORMANDO OS TIPOS DAS COLUNAS PARADA DATA


In [ ]:
df = df.withColumn("publicado_em_data", F.col("publicado_em_data").cast("date"))
df = df.withColumn("data_destaque", F.col("data_destaque").cast("date"))

df.show(5)
df.printSchema()

+-----------+--------------------+--------------------+--------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|   id_video|        titulo_video|            id_canal|          nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|coment_por_visualizacao_100|
+-----------+--------------------+--------------------+--------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCo

In [ ]:
df.printSchema()

root
 |-- id_video: string (nullable = true)
 |-- titulo_video: string (nullable = true)
 |-- id_canal: string (nullable = true)
 |-- nome_canal: string (nullable = true)
 |-- cont_visualizacao: integer (nullable = true)
 |-- curtidas: integer (nullable = true)
 |-- nao_curtidas: integer (nullable = true)
 |-- cont_comentarios: integer (nullable = true)
 |-- comentarios_desabilitados: string (nullable = true)
 |-- curtidas_desabilitadas: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- publicado_em_data: date (nullable = true)
 |-- data_destaque: date (nullable = true)
 |-- like_por_visualizacao_100: double (nullable = true)
 |-- deslike_por_visualizacao_100: double (nullable = true)
 |-- coment_por_visualizacao_100: double (nullable = true)



#RANK DE VIDEOS DENTRO DE UM CANAL


In [ ]:
#RANK DOS VÍDEOS COM MAIS CURTIDA/VISUALIZAÇÃO EM UM CANAL
w0 = Window.partitionBy(F.col("nome_canal")).orderBy("like_por_visualizacao_100")

(df.withColumn("RANK",F.rank().over(w0))
  .withColumn("curtida_por_visualizacao",F.max(F.col("like_por_visualizacao_100")).over(w0))

  .select("RANK","curtida_por_visualizacao",'titulo_video','nome_canal',"pais","publicado_em_data").show(200)
)

+----+------------------------+---------------------------------+-------------------------+----+-----------------+
|RANK|curtida_por_visualizacao|                     titulo_video|               nome_canal|pais|publicado_em_data|
+----+------------------------+---------------------------------+-------------------------+----+-----------------+
|   1|                   10.17|             #Shorts ¿Es esta ...|      PONGAMOSLO A PRU...|  MX|       2021-10-07|
|   2|                   10.18|             #Shorts ¿Es esta ...|      PONGAMOSLO A PRU...|  MX|       2021-10-07|
|   3|                    10.2|             #Shorts ¿Es esta ...|      PONGAMOSLO A PRU...|  MX|       2021-10-07|
|   4|                   10.21|             #Shorts ¿El mejor...|      PONGAMOSLO A PRU...|  MX|       2021-11-09|
|   5|                   10.22|             #Shorts ¿El mejor...|      PONGAMOSLO A PRU...|  MX|       2021-11-09|
|   6|                   10.24|             #Shorts ¿El mejor...|      PONGAMOSL

In [ ]:
#RANK DOS VÍDEOS COM MAIS DESCURTIDA/VISUALIZAÇÃO EM UM CANAL
w0 = Window.partitionBy(F.col("nome_canal")).orderBy("deslike_por_visualizacao_100")

(df.withColumn("RANK",F.row_number().over(w0))
  .withColumn("descurtida_por_visualizacao",F.max(F.col("deslike_por_visualizacao_100")).over(w0))

  .select("RANK","descurtida_por_visualizacao",'titulo_video','nome_canal',"pais","publicado_em_data").show(200)
)

+----+---------------------------+---------------------------------+-------------------------+----+-----------------+
|RANK|descurtida_por_visualizacao|                     titulo_video|               nome_canal|pais|publicado_em_data|
+----+---------------------------+---------------------------------+-------------------------+----+-----------------+
|   1|                       0.11|             #Shorts, puse a p...|      PONGAMOSLO A PRU...|  MX|       2021-08-09|
|   2|                       0.11|             #Shorts, puse a p...|      PONGAMOSLO A PRU...|  MX|       2021-08-09|
|   3|                       0.11|             #Shorts Puse a pr...|      PONGAMOSLO A PRU...|  MX|       2021-08-12|
|   4|                       0.11|             #Shorts, puse a p...|      PONGAMOSLO A PRU...|  MX|       2021-08-09|
|   5|                       0.11|             #Shorts Puse a pr...|      PONGAMOSLO A PRU...|  MX|       2021-08-12|
|   6|                       0.11|             #Shorts, 

#DESTAQUE TOP 10

In [ ]:
(df.groupBy(F.col("titulo_video")).agg(
    F.max("cont_visualizacao").alias("total_visualizacao"),
    F.max("publicado_em_data").alias("publicado_em_data"),
    F.max("data_destaque").alias("data_destaque")).orderBy('data_destaque', ascending=False).show(10)
)

+--------------------+------------------+-----------------+-------------+
|        titulo_video|total_visualizacao|publicado_em_data|data_destaque|
+--------------------+------------------+-----------------+-------------+
|GRÊMIO 3 X 0 BRAG...|            391856|       2021-11-16|   2021-11-17|
|UK VS JAMAICA FOO...|             96259|       2021-11-13|   2021-11-17|
|I GOT ENGAGED!! *...|            804866|       2021-11-14|   2021-11-17|
|This Christmas, N...|            287355|       2021-11-13|   2021-11-17|
|Oddly satisfying ...|            738795|       2021-11-15|   2021-11-17|
|Anitta - Envolver...|           5087138|       2021-11-12|   2021-11-17|
|HYPE! en attendan...|             45617|       2021-11-13|   2021-11-17|
|Voyage - Uloge (O...|            803687|       2021-11-16|   2021-11-17|
|HAUT PARLEUR pass...|             91737|       2021-11-13|   2021-11-17|
|La Palma: Vulkang...|            165681|       2021-11-11|   2021-11-17|
+--------------------+----------------

TOP 10 MUNDO

In [ ]:
(df.groupBy(F.col("nome_canal")).agg(
    F.sum("cont_visualizacao").alias("soma_visualizacao"),
    F.round(F.avg("cont_visualizacao"),2).alias("media_visualizacao"),
    F.sum("curtidas").alias("soma_curtidas"),
    F.round(F.avg("curtidas"),2).alias("media_curtidas"),
    F.sum("nao_curtidas").alias("soma_naocurtidas"),
    F.max("publicado_em_data").alias("publicado_em_data"),
    F.max("data_destaque").alias("data_destaque"),
    F.round(F.avg("nao_curtidas"),2).alias("media_naocurtidas")).orderBy('soma_visualizacao', ascending=False).show(10)
)





+-----------------+-----------------+------------------+-------------+--------------+----------------+-----------------+-------------+-----------------+
|       nome_canal|soma_visualizacao|media_visualizacao|soma_curtidas|media_curtidas|soma_naocurtidas|publicado_em_data|data_destaque|media_naocurtidas|
+-----------------+-----------------+------------------+-------------+--------------+----------------+-----------------+-------------+-----------------+
|        BLACKPINK|      58230237271|      2.82396883E7|   5596684546|    2714202.01|       117843253|       2021-10-28|   2021-11-01|         57149.98|
|      HYBE LABELS|      47383301434|     3.197253808E7|   4954981634|     3343442.4|        30903181|       2021-10-27|   2021-11-04|         20852.35|
|           SMTOWN|      46008908926|     2.024149095E7|   2845074443|    1251682.55|        40529445|       2021-10-25|   2021-11-06|         17830.82|
|JYP Entertainment|      38480556253|     1.798156834E7|   2296325821|    1073049.

#PARABENS ANITTA 👌👌👌

In [ ]:
df.where((F.col("pais")=='BR')).filter(" like_por_visualizacao_100  > 50").show(20)


+-----------+--------------------+--------------------+----------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|   id_video|        titulo_video|            id_canal|nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|coment_por_visualizacao_100|
+-----------+--------------------+--------------------+----------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|CuyTC8FLICY|Anitta - Girl Fro...|UCqjjyPUghDSSKFBA...|    Anitta|           544091|  32564

#SALVANDO NO BUCKET

In [ ]:
# printando o schema só para ver todos os tipos das colunas
#df.printSchema()

In [ ]:
# dropando a colunas de 'virou_destaque_em_dias', pois estava dando problema para salvar. Tentem salvar com ela para ver o erro!
# coloquei em dataframe novo para poderem testar
#df2 = df.drop('virou_destaque_em_dias')
#df2.show(1)

Dessa maneira, salvar usando o toPandas não deu certo.

In [ ]:
#serviceAccount = '/content/projetofinalgrupo8-2dcd866c3f46.json'
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount
#
#client = storage.Client()
#bucket = client.get_bucket('projetofinalgrupo8')

#bucket.blob('saida/data_tratado_pyspark.csv').upload_from_string(df2.to_csv(index=False), 'text/csv')
#bucket.blob('saida/data_tratado_pyspark.csv').upload_from_string(df.toPandas().to_csv(index=False), 'text/csv')

Dessa forma funciona, é um salvamento nativo do databricks, na raiz da aba lateral do colab que ele salva com propriedade de repartir arquivos grandes, isso acontece com o nosso.

In [ ]:
df.write.format("com.databricks.spark.csv").option("header", "true")\
                                           .option("inferschema", "true")\
                                           .option("delimiter", ",")\
                                           .save("/content/data_tratado_pyspark.csv")

Aqui foi carreguado os as duas partes do nosso banco de dados que foi repartido no salvamento anterior

In [ ]:
#COPIAR OS ENDEREÇOS DOS ARQUIVOS NA PASTA "data_tratado_pyspark.csv"
df_pandas1 = pd.read_csv('/content/data_tratado_pyspark.csv/part-00000-445148dc-72ca-4212-9367-c6c5c5398542-c000.csv')
df_pandas2 = pd.read_csv('/content/data_tratado_pyspark.csv/part-00001-445148dc-72ca-4212-9367-c6c5c5398542-c000.csv')
df_pandas2.head(1)

,id_video,titulo_video,id_canal,nome_canal,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria,publicado_em_data,data_destaque,like_por_visualizacao_100,deslike_por_visualizacao_100,coment_por_visualizacao_100
0,qDNssUZPZmw,Teri Aadat - Behind The Scenes ft. Siddharth N...,UCoFxUkg6cLuhlY3Ll8SHTDQ,Anushka Sen,254908,23129,629,1443,False,False,IN,entretenimento,2021-03-26,2021-03-28,9.07,0.25,0.57


Curiosamente ambos ficaram com tamanho bem proximo --> (510976, 17)

In [ ]:
print(df_pandas1.shape)
print(df_pandas2.shape)

(510976, 17)
(510645, 17)


Concatenação e verificação do tamanho, deu tudo certo

In [ ]:
df_pandas_def = pd.concat([df_pandas1, df_pandas2])
df_pandas_def.shape

(1021621, 17)


Por fim, foi possivel salvar no bucket ✌

In [ ]:
serviceAccount = '/content/projetofinalgrupo8-2dcd866c3f46.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

client = storage.Client()
bucket = client.get_bucket('projetofinalgrupo8')
bucket.blob('saida/data_tratado_pyspark.csv').upload_from_string(df_pandas_def.to_csv(index=False), 'text/csv')